<a href="https://colab.research.google.com/github/ShadaFathima/ML-DL/blob/main/CNN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
# data_dir = '/content/drive/MyDrive/MGU_Workshop/Day1-FN/new'
data_dir = '/content/drive/MyDrive/DL/covidData'
img_size = 224
batch_size = 16
num_epochs = 100
val_split = 0.2
seed = 42

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(seed)
np.random.seed(seed)


In [ ]:
# ─── Data Loading ───────────────────────────────────────────────────────
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [ ]:

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes
num_classes = len(class_names)


In [ ]:
val_size = int(val_split * len(full_dataset))
train_size = len(full_dataset) - val_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])


In [ ]:

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, pin_memory=True)



In [ ]:

# ─── CNN Model ─────────────────────────────────────────────────────────
class ChestXRayCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 256),
            nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        return self.classifier(x)


model = ChestXRayCNN(num_classes).to(device)


In [ ]:


# ─── Training Setup ─────────────────────────────────────────────────────
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_val_acc = 0.0


In [ ]:

# ─── Training Loop ─────────────────────────────────────────────────────
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for phase in ['train', 'val']:
        model.train() if phase == 'train' else model.eval()
        loader = train_loader if phase == 'train' else val_loader

        running_loss = 0.0
        correct_preds = 0

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                preds = torch.argmax(outputs, 1)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct_preds += torch.sum(preds == labels)

        epoch_loss = running_loss / len(loader.dataset)
        epoch_acc = correct_preds.double() / len(loader.dataset)
        print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

        if phase == 'val' and epoch_acc > best_val_acc:
            best_val_acc = epoch_acc
            best_model_state = model.state_dict().copy()


Epoch 1/100


In [ ]:
# Load best weights
model.load_state_dict(best_model_state)

In [ ]:
# ─── Evaluation ─────────────────────────────────────────────────────────
model.eval()
all_preds, all_labels = [], []

In [ ]:
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

In [ ]:
# ─── Visualization ─────────────────────────────────────────────────────
def imshow(img_tensor, title):
    img = img_tensor.squeeze().cpu().numpy()
    img = img * 0.5 + 0.5  # Unnormalize
    plt.imshow(img, cmap="gray")
    plt.title(title)
    plt.axis('off')

In [ ]:
inputs, labels = next(iter(val_loader))
inputs, labels = inputs.to(device), labels.to(device)
outputs = model(inputs)
preds = torch.argmax(outputs, 1)

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i+1)
    title = f"T: {class_names[labels[i]]} | P: {class_names[preds[i]]}"
    imshow(inputs[i], title)
plt.tight_layout()
plt.show()

In [ ]:
# ─── Predict a Single Image ────────────────────────────────────────────
def predict_image(image_path, model, transform):
    model.eval()
    img = Image.open(image_path).convert('L')
    img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, 1)
    return class_names[pred.item()]

In [ ]:
# Usage:
#result = predict_image('/content/drive/MyDrive/MGU_Workshop/Day1-FN/new/Covid/03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg', model, transform)
result = predict_image('//content/drive/MyDrive/DL/covidData/PNEUMONIA/person142_bacteria_683.jpeg',model, transform)
print("Prediction:", result)